### Untitled

###### Web Automation

In [ ]:
import numpy as np
import pandas as pd
import re
import time

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

xpath1 = '//*[@id="ypgBody"]/div[2]/div/div[1]/div[10]/div[2]/a'
xpath2 = '//*[@id="ypgBody"]/div[2]/div/div[1]/div[10]/div[2]/a[2]'
xpaths = [xpath1,xpath2]
xpaths.extend([xpath2 for i in range(47)])

url = 'https://www.yellowpages.ca/search/si-alph/1/Coffee+Shops/Downtown+Toronto+Toronto+ON'

In [ ]:
def crawl_dir(url):
    
    '''
    This is a function to parse the online directory based on the url that is passed
    '''
    driver = webdriver.Chrome("/Users/TaylorMK23/Downloads/chromedriver")
    driver.get(url)
    for i,xpath in enumerate(xpaths):

        content = driver.page_source
        soup = BeautifulSoup(content)

        for d in soup.findAll('div', attrs={'class':'listing__content__wrap--flexed jsGoToMp'}):
                try:
                    name = d.find('a', attrs={'class':'listing__name--link listing__link jsListingName'})
                    shop_name.append(name.text)
                except AttributeError:
                    name_text = 'NaN'
                    shop_name.append(name_text)
                try:
                    street = d.find('span', attrs={'itemprop':'streetAddress'})
                    streetAddress.append(street)
                except AttributeError:
                    street_text = 'NaN'
                    streetAddress.append(street_text)
                try:
                    locality = d.find('span', attrs={'itemprop':'addressLocality'})
                    addressLocality.append(locality)
                except AttributeError:
                    addressLocality.append('NaN')
                try:
                    region = d.find('span', attrs={'itemprop':'addressRegion'})
                    addressRegion.append(region)
                except AttributeError:
                    addressRegion.append('NaN')
                try:
                    postal = d.find('span', attrs={'itemprop':'postalCode'})
                    postalCode.append(postal)
                except AttributeError:
                    postalCode.append('NaN')

        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath))).click()
        url = driver.current_url
        driver.get(url)
        time.sleep(10)
    

In [ ]:
# crawl_dir(url)

In [ ]:
def clean_list(streetAddress,addressLocality,addressRegion,postalCode,lst1,lst2,lst3,lst4,full_address):
        
    for i in streetAddress:
        i=str(i)
        result = i.find('<')
        result2 = i.find('>')
        result3 = i.find('<',result+1)
        street = i[result2+1:result3]
        lst1.append(street + ', ')
        
    for i in addressLocality:
        i=str(i)
        result = i.find('<')
        result2 = i.find('>')
        result3 = i.find('<',result+1)
        locality = i[result2+1:result3]
        lst2.append(locality + ', ')
        
    for i in addressRegion:
        i=str(i)
        result = i.find('<')
        result2 = i.find('>')
        result3 = i.find('<',result+1)
        region = i[result2+1:result3]
        lst3.append(region + ', ')
        
    for i in postalCode:
        i=str(i)
        result = i.find('<')
        result2 = i.find('>')
        result3 = i.find('<',result+1)
        postal = i[result2+1:result3]
        lst4.append(postal)
        

In [ ]:
lst1 = []
lst2 = []
lst3 = []
lst4 = []
full_address = []
clean_list(streetAddress,addressLocality,addressRegion,postalCode,lst1,lst2,lst3,lst4,full_address)

###### Save to CSV and Create a DataFrame

In [ ]:
# DataFrame.to_csv('data.coffee_shops.csv')

In [32]:
df = pd.DataFrame({'shop_name':shop_name,'STREET':lst1,'LOCALITY':lst2,'REGION':lst3,'POSTAL_CODE':lst4})
df.T

NameError: name 'shop_name' is not defined

In [33]:
df = pd.read_csv('data/coffee_shops.csv')

In [34]:
df['address']=df['STREET'] + df['LOCALITY'] + df['REGION'] + df['POSTAL_CODE']

In [35]:
df.drop(['STREET','LOCALITY','REGION','POSTAL_CODE'], axis=1, inplace=True)
df.describe()

,Unnamed: 0
count,1680.000000
mean,839.500000
std,485.118542
min,0.000000
25%,419.750000
50%,839.500000
75%,1259.250000
max,1679.000000


###### Data Cleaning 

In [36]:
duplicates = df[df.duplicated(['address'])]
duplicates.tail(3)

,Unnamed: 0,shop_name,address
1656,1656,Voulez Vous Café,"Non, Non, ON, Non"
1666,1666,Wilson Cafe,"620 Wilson Ave, North York, ON, M3K 1Z3"
1668,1668,Yogen Früz,"2300 Yonge St, Toronto, ON, M4P 1E4"


In [37]:
print(df['shop_name'].nunique())
print(df['shop_name'].unique())


789
['0109 Dessert And Cafe' '1993530 Ontario Inc' '2-4-1 Pizza'
 '3 Kings Cafe' '416 Coffee' '5 Elements Espresso' '521 Cafe & Bar'
 '608 Design' '650 Cafe Bistro - Downtown' '709 Greet Coffee'
 '836 Pape Coffee Shop' 'A-Game Cafe' 'Abba Kyrillos Mari Mina'
 'ABC Bakery & Coffee Shop' 'Ada Chocolate' "Adam's Cafe" 'Addis Cafe'
 'Affluent Minda' 'Afpetta Caffe' 'Akia King Cafe' 'Al-Jandoul Group Ltd.'
 'Alcove Coffee' 'Allstarz Expresso Bar' 'Alumni Cafe' 'Amani Earls'
 'Amazon Coffee Roasters Ltd' 'Ambiance Chocolat' "Amr Foods-Roody's Cafe"
 'Ana-Sam Investments Inc' 'Anat Buna Coffee Shop' "Andy's"
 "Andy's Coffee Shop" 'Anondo Resturant' "Anthony's Cafe" "Antonio's Cafe"
 'Aramark Canada Ltd' 'Archtop Cafe & Music Ltd' 'Argila Cafe'
 'Aroma Espresso' 'Aroma Espresso Bar' 'Aroma Espresso Bar Canada Inc'
 'Aroma Espresso Bar Waterpark Place' 'Aroma Expresso'
 'Aroma Expresso Bar' 'Aroma Expresso Bar Canada Inc'
 'Arte Gelato And Cafe' 'Artisano Bakery Cafe Ltd'
 'Artisano Bakery Café

*First we make all letters in the names into lowercase, this would eliminate some of the discrepancies easily*

In [38]:
df['shop_name'] = df['shop_name'].str.lower()
print(df['shop_name'].nunique())
print(df['shop_name'].unique())

786
['0109 dessert and cafe' '1993530 ontario inc' '2-4-1 pizza'
 '3 kings cafe' '416 coffee' '5 elements espresso' '521 cafe & bar'
 '608 design' '650 cafe bistro - downtown' '709 greet coffee'
 '836 pape coffee shop' 'a-game cafe' 'abba kyrillos mari mina'
 'abc bakery & coffee shop' 'ada chocolate' "adam's cafe" 'addis cafe'
 'affluent minda' 'afpetta caffe' 'akia king cafe' 'al-jandoul group ltd.'
 'alcove coffee' 'allstarz expresso bar' 'alumni cafe' 'amani earls'
 'amazon coffee roasters ltd' 'ambiance chocolat' "amr foods-roody's cafe"
 'ana-sam investments inc' 'anat buna coffee shop' "andy's"
 "andy's coffee shop" 'anondo resturant' "anthony's cafe" "antonio's cafe"
 'aramark canada ltd' 'archtop cafe & music ltd' 'argila cafe'
 'aroma espresso' 'aroma espresso bar' 'aroma espresso bar canada inc'
 'aroma espresso bar waterpark place' 'aroma expresso'
 'aroma expresso bar' 'aroma expresso bar canada inc'
 'arte gelato and cafe' 'artisano bakery cafe ltd'
 'artisano bakery café

*We notice a lot of the Coffee Shops are misspelled we need to clean this before proceeding*

*Correcting known problematic words such as Expresso, Coffe, Cofee*

##### *Following the Pareto Principle, majority of the mislabelled Coffee Shops can be fixed by focusing on just a few, the big name chains like Starbucks, Tim Hortons, Timothy's and Aroma Espresso Bar*
*This would be done iteratively to keep improving the results*

In [39]:
import difflib
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [40]:
big_chains = ["aroma espresso bar", "balzac's ltd", "second coffee co.", 
              "starbucks", "timothy's world", "tim hortons"]
# unique_names = list(df['shop_name'].unique())
unique_names = list(df['shop_name'])

In [41]:
def use_similar(correct_names,full_list, threshold):

    for a,x in enumerate(full_list):
        for b,y in enumerate(correct_names):

            if similar(correct_names[b],full_list[a]) >= threshold:
                full_list[a] = correct_names[b]
            elif similar(correct_names[b],full_list[a]) < threshold:
                full_list[a] = full_list[a]
    return unique_names

In [42]:
use_similar(big_chains,unique_names, 0.75)[700:]

["peter's cafe",
 'petit nuage',
 'petit paris cake and coffee shop toronto',
 'phipps bakery cafe',
 'pho asia 39',
 'photo cafe',
 'phyllo you bakery',
 'pico sports bar & cafe',
 'pilot coffee',
 'pinar del rio espresso bar',
 'pirgos coffee shop',
 'pizza thick & the ice cream bake shop',
 'play caffe & bar',
 'play creative café',
 'plentea',
 'poetry jazz cafe',
 'poets and pirates cafe',
 'pop up house',
 'press cafe',
 'presse café',
 'presse cafe',
 'presse café',
 'presse cafe',
 'aroma espresso bar',
 'pro stop sports bar & café inc',
 'pronto cafe',
 'punch buggy 5',
 'purple penguin café',
 'quarry cafe',
 'quattro amci cafe',
 'queen b pastry',
 'queen mother cafe',
 'queen mother cafe',
 'queen street general store & cafe',
 "queen's pasta cafe",
 'quirks cafe',
 'quorum café',
 "rachel's coffee house",
 'radio station cafe',
 "ralphy's corner cafe",
 'red rocket coffee',
 'red rocket coffee',
 'red rocket coffee',
 'red seal masonry',
 'renaissance cafe',
 'renaissance 

In [43]:
def rename(lst):
    for (i,v) in enumerate(lst):
        if v == "tim hortons - closed":
            lst[i] = "tim hortons"
        elif v == "tim hortons - temporarily closed":
            lst[i] = "tim hortons"
        elif v == "second cup coffee co. - closed":
            lst[i] = "second coffee co."
        elif v == "second cup coffee co. featuring pinkberry frozen yogurt":
            lst[i] = "second coffee co."
        elif v == "second cup":
            lst[i] = "second coffee co."
        elif v == "country style donuts":
            lst[i] = "country style"
        elif v == "country style petroplus":
            lst[i] = "country style"
        elif v == "express country style":
            lst[i] = "country style"
    return lst

In [44]:
rename(unique_names)[700:]

["peter's cafe",
 'petit nuage',
 'petit paris cake and coffee shop toronto',
 'phipps bakery cafe',
 'pho asia 39',
 'photo cafe',
 'phyllo you bakery',
 'pico sports bar & cafe',
 'pilot coffee',
 'pinar del rio espresso bar',
 'pirgos coffee shop',
 'pizza thick & the ice cream bake shop',
 'play caffe & bar',
 'play creative café',
 'plentea',
 'poetry jazz cafe',
 'poets and pirates cafe',
 'pop up house',
 'press cafe',
 'presse café',
 'presse cafe',
 'presse café',
 'presse cafe',
 'aroma espresso bar',
 'pro stop sports bar & café inc',
 'pronto cafe',
 'punch buggy 5',
 'purple penguin café',
 'quarry cafe',
 'quattro amci cafe',
 'queen b pastry',
 'queen mother cafe',
 'queen mother cafe',
 'queen street general store & cafe',
 "queen's pasta cafe",
 'quirks cafe',
 'quorum café',
 "rachel's coffee house",
 'radio station cafe',
 "ralphy's corner cafe",
 'red rocket coffee',
 'red rocket coffee',
 'red rocket coffee',
 'red seal masonry',
 'renaissance cafe',
 'renaissance 

In [45]:
unique_names[-6:-1] = ["zaza espresso bar"]*5

In [46]:
len(unique_names)

1680

In [47]:
df['shop_name'] = unique_names
df.head()

,Unnamed: 0,shop_name,address
0,0,0109 dessert and cafe,"2190 McNicoll Ave, Scarborough, ON, M1V 0B3"
1,1,1993530 ontario inc,"5150 Yonge St, North York, ON, M2N 6L8"
2,2,2-4-1 pizza,"2834 Victoria Park Ave, North York, ON, M2J 4A8"
3,3,3 kings cafe,"2061 Lawrence Ave E, Scarborough, ON, M1R 2Z4"
4,4,416 coffee,"2-36 Wright St, St Catharines, ON, L2P 3J4"


###### Load Libraries

In [118]:
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import geopy
import geocoder
from mapboxgl.viz import CircleViz
from mapboxgl.viz import MapViz
import mapboxgl
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from shapely.geometry import Polygon, LineString, Point, box
import pandana
from pandana import Network
import warnings
warnings.filterwarnings('ignore')

In [49]:
mapbox_access_token = 'pk.eyJ1Ijoia2Fuam9tZWxvIiwiYSI6ImNrZWJ6aWM2NzBkcmwzM3FsZXZzcWhxbWMifQ.E-yHj-_8YVnf-KD2-zP0oA'

In [50]:
# sample test of geocoder with first result from the dataframe
g = geocoder.mapbox('Ontario Food Terminal, Etobicoke, ON M8Y 1H8', key='pk.eyJ1Ijoia2Fuam9tZWxvIiwiYSI6ImNrZWJ6aWM2NzBkcmwzM3FsZXZzcWhxbWMifQ.E-yHj-_8YVnf-KD2-zP0oA')
print(g.lat,g.lng)

43.628714 -79.4856075


In [51]:
lat = []
lng = []
for i,v in enumerate(list(df['address'])):
    g = geocoder.mapbox(v, key=mapbox_access_token)
    lat.append(g.lat)
    lng.append(g.lng)

len(lat)

Status code 404 from https://api.mapbox.com/geocoding/v5/mapbox.places/1966 1/2 QueenE, Toronto, ON, M4L 1H8.json: ERROR - 404 Client Error: Not Found for url: https://api.mapbox.com/geocoding/v5/mapbox.places/1966%201/2%20QueenE,%20Toronto,%20ON,%20M4L%201H8.json?access_token=pk.eyJ1Ijoia2Fuam9tZWxvIiwiYSI6ImNrZWJ6aWM2NzBkcmwzM3FsZXZzcWhxbWMifQ.E-yHj-_8YVnf-KD2-zP0oA
Status code 404 from https://api.mapbox.com/geocoding/v5/mapbox.places/1530 Albion Rd., Unit #61, Etobicoke, ON, M9V 1B4.json: ERROR - 404 Client Error: Not Found for url: https://api.mapbox.com/geocoding/v5/mapbox.places/1530%20Albion%20Rd.,%20Unit%20?access_token=pk.eyJ1Ijoia2Fuam9tZWxvIiwiYSI6ImNrZWJ6aWM2NzBkcmwzM3FsZXZzcWhxbWMifQ.E-yHj-_8YVnf-KD2-zP0oA#61,%20Etobicoke,%20ON,%20M9V%201B4.json
Status code 403 from https://api.mapbox.com/geocoding/v5/mapbox.places/2825 Markham Rd &amp; Mcnicoll Ave, Toronto, ON, M1B 2W1.json: ERROR - 403 Client Error: Forbidden for url: https://api.mapbox.com/geocoding/v5/mapbox.places/2

1680

In [52]:
df.isna().sum()

Unnamed: 0    0
shop_name     0
address       0
dtype: int64

In [53]:
df.dropna(how='any',inplace=True)
df.shape

(1680, 3)

In [54]:
df['lat']=lat
df['lng']=lng

In [55]:
df.head(2)

,Unnamed: 0,shop_name,address,lat,lng
0,0,0109 dessert and cafe,"2190 McNicoll Ave, Scarborough, ON, M1V 0B3",43.814767,-79.294908
1,1,1993530 ontario inc,"5150 Yonge St, North York, ON, M2N 6L8",43.768870,-79.413067


###### Creating Unique Ids for the Coffee Shops

In [56]:
# create a list of unique names
names = df['shop_name'].unique().tolist()

# generte ids
ids = np.random.randint(low=1e1, high=1e10, size = len(names))

# maps ids to names
maps = {k:v for k,v in zip(names, ids)}

# add new id column
df['id'] = df['shop_name'].map(maps)

###### Create x and y Geometries for distance calculations later on

In [103]:
df = df2
cond1 = df['lat'] < 46
cond2 = df['lng'] < -78
cond3 = df['lat'] > 42
cond4 = df['lng'] > -79.5
df = df[cond1 & cond2 & cond3 & cond4]

In [104]:
df['lat'].max()

44.390731

In [106]:
df.describe()

,Unnamed: 0,lat,lng,id,x,y
count,1501.000000,1501.000000,1501.000000,1.501000e+03,1501.000000,1501.000000
mean,838.500333,43.696559,-79.381764,5.964231e+09,-79.381764,43.696559
std,484.682993,0.066729,0.065841,3.090839e+09,0.065841,0.066729
min,0.000000,43.166340,-79.499223,1.204810e+06,-79.499223,43.166340
25%,420.000000,43.653759,-79.413346,2.940119e+09,-79.413346,43.653759
50%,841.000000,43.670516,-79.388212,7.083296e+09,-79.388212,43.670516
75%,1250.000000,43.725575,-79.359662,8.040896e+09,-79.359662,43.725575
max,1679.000000,44.390731,-78.380113,9.996407e+09,-78.380113,44.390731


In [107]:
# create geopanda dataframe add crs lat long
df['geometry'] = list(zip(df.lng, df.lat))
df['geometry'] = df['geometry'].apply(Point)

# reprojecting epsg 4386 (wgs84) to epsg 2019 (mtm nad 27)
crs = {'init': 'epsg:4326'}
df = gpd.GeoDataFrame(df, crs=crs, geometry='geometry')
df = df.to_crs({'init': 'epsg:2019'})

# calculating x and y which will be used later to find nearest ttc stop to address 
df['x'] = df.geometry.x
df['y'] = df.geometry.y

In [108]:
df.tail()

,Unnamed: 0,shop_name,address,lat,lng,id,geometry,x,y
1673,1673,zav coffee shop & gallery,"2048 Danforth Ave, Toronto, ON, M4C 1J6",43.685785,-79.314008,140015307,POINT (319795.144 4838227.220),319795.143945,4.838227e+06
1674,1674,zaza espresso bar,"806 Lawrence Ave W, North York, ON, M6A 0B1",43.714487,-79.453897,2895027568,POINT (308515.167 4841400.066),308515.167181,4.841400e+06
1675,1675,zaza espresso bar,"258 Beresford Ave, Toronto, ON, M6S 3B3",43.651021,-79.478062,2895027568,POINT (306569.718 4834348.590),306569.717770,4.834349e+06
1677,1677,zaza espresso bar,"775 St Clair Ave W, Toronto, ON, M6C 1B7",43.680736,-79.428944,2895027568,POINT (310529.194 4837651.951),310529.193896,4.837652e+06
1679,1679,zooli's inc,"1 Bedford, Toronto, ON, M5R 2B5",43.668772,-79.397518,2470901880,POINT (313064.691 4836325.477),313064.691363,4.836325e+06


In [109]:
# with_loc = df.to_csv('data/with_latlon.csv')
# with_loc2 = gdf.to_csv('data/with_latlon2.csv')

In [4]:
df = pd.read_csv('data/with_latlon2.csv')


In [6]:
type(df)

pandas.core.frame.DataFrame

###### Read the Address File

In [130]:
zipfile = "zip://data/municipal-address-points-wgs84-latitude-longitude.zip"
address = gpd.read_file(zipfile)

In [131]:
type(address)

geopandas.geodataframe.GeoDataFrame

In [132]:
address.columns = map(str.lower, address.columns)

#include columns needed for the project
address = address[['geo_id', 'maint_stag','address','lfname','x','y','longitude','latitude','objectid','mun_name','ward_name','geometry']]
address.head(2)

,geo_id,maint_stag,address,lfname,x,y,longitude,latitude,objectid,mun_name,ward_name,geometry
0,1015200,REGULAR,22,Lloyd George Ave,301211.152,4827985.051,-79.544251,43.593789,4104309.0,Etobicoke,Etobicoke-Lakeshore,POINT (-79.54425 43.59379)
1,1015189,REGULAR,3,Lloyd George Ave,301272.174,4827946.759,-79.543495,43.593445,4104306.0,Etobicoke,Etobicoke-Lakeshore,POINT (-79.54350 43.59344)


In [ ]:
crs = {'init': 'epsg:4326'}
address = gpd.GeoDataFrame(address, crs=crs, geometry='geometry')
address = address.to_crs({'init': 'epsg:2019'})
ax = address.plot(figsize=(15, 15),color='blue', markersize =1)


###### Creating the Pedestrian Network 

In [10]:
pednet = gpd.read_file("zip://data/pednet.zip")

In [11]:
# reprojecting epsg 4386 (wgs84) to epsg 2019 (mtm nad 27)
crs = {'init': 'epsg:4326'}
pednet = gpd.GeoDataFrame(pednet, crs=crs, geometry='geometry')
pednet = pednet.to_crs({'init': 'epsg:2019'})

# remove length column to avoid confusion
pednet = pednet[['OBJECTID', 'road_type', 'sdwlk_code', 'sdwlk_desc', 'crosswalk', 'cwalk_type', 'px', 'px_type','geometry']]
pednet.head(2)

,OBJECTID,road_type,sdwlk_code,sdwlk_desc,crosswalk,cwalk_type,px,px_type,geometry
0,1,Local,7.0,Sidewalk on both sides,0,None,None,None,"LINESTRING (299648.279 4843992.545, 299731.442..."
1,2,Collector,7.0,Sidewalk on both sides,0,None,None,None,"LINESTRING (299320.274 4832700.973, 299352.702..."


In [12]:
# creating network graph code
def create_graph(gdf, precision=3):
    '''Create a networkx given a GeoDataFrame of lines. Every line will
    correspond to two directional graph edges, one forward, one reverse. The
    original line row and direction will be stored in each edge. Every node
    will be where endpoints meet (determined by being very close together) and
    will store a clockwise ordering of incoming edges.
    '''
      
    G = nx.Graph()

    def make_node(coord, precision):
        return tuple(np.round(coord, precision))

    # Edges are stored as (from, to, data), where from and to are nodes.
    def add_edges(row, G):
        geometry = row.geometry
        coords = list(geometry.coords)
        geom_r = LineString(coords[::-1])
        coords_r = geom_r.coords
        start = make_node(coords[0], precision)
        end = make_node(coords[-1], precision)
        # Add forward edge
        fwd_attr ={}
        for k,v in row.items():
            fwd_attr[k]=v
        fwd_attr['forward']= 1
       #fwd_attr['geometry']=  geometry
        fwd_attr['length']=  geometry.length

        fwd_attr['visited']= 0

        G.add_edge(start, end, **fwd_attr)

    gdf.apply(add_edges, axis=1, args=[G])

    return G

In [13]:
# creating network graph
G = create_graph(pednet)

In [14]:
# get network "from" and "to" from nodes
edges = nx.to_pandas_edgelist(G,'from','to')
to = edges['to'].tolist()
fr = edges['from'].tolist()
fr = list(set(fr))
to = list(set(to))
to.extend(fr)
nodes = list(set(to))
nodes = pd.DataFrame(nodes)
nodes.columns=['x', 'y']
nodes['xy'] = nodes.apply(lambda z: (z.x,z.y),axis=1)

In [15]:
# Assigning node ids to to_node and from_node

nodes['id'] = nodes.index
edges['to_node']=edges['to'].map(nodes.set_index('xy').id)
edges['from_node']=edges['from'].map(nodes.set_index('xy').id)
edges.head(3)

,from,to,px,forward,cwalk_type,sdwlk_code,geometry,road_type,visited,px_type,crosswalk,OBJECTID,length,sdwlk_desc,to_node,from_node
0,"(299648.279, 4843992.545)","(299731.442, 4843949.007)",None,1,None,7.0,LINESTRING (299648.2789610377 4843992.54504967...,Local,0,None,0,1,93.870579,Sidewalk on both sides,51829,56074
1,"(299648.279, 4843992.545)","(299603.437, 4844062.355)",None,1,None,7.0,LINESTRING (299603.437224735 4844062.355196978...,Local,0,None,0,84099,82.971308,Sidewalk on both sides,48292,56074
2,"(299731.442, 4843949.007)","(299812.731, 4843907.011)",None,1,None,3.0,LINESTRING (299731.4420138962 4843949.00664981...,Local,0,None,0,25712,91.495742,No sidewalk on either side,42948,51829


In [16]:
# creating pandana network

transit_ped_net = pdna.Network(nodes["x"],
                               nodes["y"],
                               edges["from_node"],
                               edges["to_node"],
                               pd.DataFrame([edges['length']]).T,
                               twoway=True)

# saving walkability file is optional. It can be used in the next steps if you don't have transit_ped_net in memory
transit_ped_net.save_hdf5('./walkability.hd5')

In [17]:
transit_ped_net = pandana.Network.from_hdf5('walkability.hd5')

In [110]:
df.dropna(how='any',inplace=True)

In [111]:
df.isna().sum()

Unnamed: 0    0
shop_name     0
address       0
lat           0
lng           0
id            0
geometry      0
x             0
y             0
dtype: int64

In [112]:
#precompute a given horizon distance of 1000 meters
transit_ped_net.precompute(1000)
transit_ped_net.edges_df.head(2)

,from,to,length
0,56074,51829,93.870579
1,56074,48292,82.971308


In [116]:
df.head(1)

,Unnamed: 0,shop_name,address,lat,lng,id,geometry,x,y,node_ids
0,0,0109 dessert and cafe,"2190 McNicoll Ave, Scarborough, ON, M1V 0B3",43.814767,-79.294908,8461923614,POINT (321299.565 4852560.122),321299.564939,4.852560e+06,38345


In [121]:
gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.lng, df.lat))
gdf.head()

,Unnamed: 0,shop_name,address,lat,lng,id,geometry,x,y,node_ids
0,0,0109 dessert and cafe,"2190 McNicoll Ave, Scarborough, ON, M1V 0B3",43.814767,-79.294908,8461923614,POINT (-79.29491 43.81477),321299.564939,4.852560e+06,38345
1,1,1993530 ontario inc,"5150 Yonge St, North York, ON, M2N 6L8",43.768870,-79.413067,1664813656,POINT (-79.41307 43.76887),311799.076644,4.847444e+06,60885
2,2,2-4-1 pizza,"2834 Victoria Park Ave, North York, ON, M2J 4A8",43.788605,-79.329420,217551067,POINT (-79.32942 43.78860),318529.078134,4.849647e+06,41648
3,3,3 kings cafe,"2061 Lawrence Ave E, Scarborough, ON, M1R 2Z4",43.745149,-79.294066,1887108790,POINT (-79.29407 43.74515),321386.537230,4.844826e+06,52124
4,4,416 coffee,"2-36 Wright St, St Catharines, ON, L2P 3J4",43.166340,-79.218052,2838632795,POINT (-79.21805 43.16634),327726.552515,4.780545e+06,21832


In [138]:
#get node_ids for points for each layer
x, y = df.x, df.y 
df["node_ids"] = transit_ped_net.get_node_ids(x, y)
transit_ped_net.set(gdf["node_ids"], name="df")

# using x and y coordinates from address that are utm nad27 
x, y = address.x, address.y
address["node_ids"] = transit_ped_net.get_node_ids(x, y)

###### Get the 3 nearest Coffee Shops

In [139]:
#get nearest points with id
n=3
maxdistance = 5000

transit_ped_net.set_pois("df", maxdistance , n, df.x, df.y)
coffee_walk_distances = transit_ped_net.nearest_pois(maxdistance , "df", num_pois=n, include_poi_ids=False)


columns =  ['coffee_'+str(i) for i in range(0,n,1)]

coffee_walk_distances.columns = columns

coffee_walk_distances.head(2)


,coffee_0,coffee_1,coffee_2
0,3213.773926,3861.658936,4025.719971
1,808.479980,808.479980,1013.148987


In [140]:
for i in range(3):
    address['coffee_{}'.format(i)]=address['node_ids'].map(coffee_walk_distances['coffee_{}'.format(i)])
address.head(3)

,geo_id,maint_stag,address,lfname,x,y,longitude,latitude,objectid,mun_name,ward_name,geometry,node_ids,coffee_0,coffee_1,coffee_2
0,1015200,REGULAR,22,Lloyd George Ave,301211.152,4827985.051,-79.544251,43.593789,4104309.0,Etobicoke,Etobicoke-Lakeshore,POINT (-79.54425 43.59379),1913,5000.0,5000.0,5000.0
1,1015189,REGULAR,3,Lloyd George Ave,301272.174,4827946.759,-79.543495,43.593445,4104306.0,Etobicoke,Etobicoke-Lakeshore,POINT (-79.54350 43.59344),14935,5000.0,5000.0,5000.0
2,20140046,REGULAR,7A,Lloyd George Ave,301265.553,4827966.490,-79.543577,43.593622,4104315.0,Etobicoke,Etobicoke-Lakeshore,POINT (-79.54358 43.59362),14935,5000.0,5000.0,5000.0


###### Color coding the shops by Company

In [ ]:
gdf = df.rename({'lat':'latitude','lng':'longitude'}, axis=1)
# gdf.drop(['Unnamed: 0','Unnamed: 0.1'], axis = 1, inplace=True)
gdf.head(3)

In [ ]:
help(gdf.plot)

In [ ]:
df2 = df.drop(['id','x','y','latitude','longitude','node_ids'], axis =1)
df2.head(1)

In [ ]:
# Walking Times To Closest Coffee Shop
ax = address.plot(figsize=(15, 15), column='coffee_0', scheme='quantiles', legend=True,markersize = 1)
plt.title('Walking Times To Closest Coffee Shop')

In [ ]:
ax = address.plot(figsize=(15, 15), column='coffee_0', scheme='quantiles', legend=True,markersize = 1)
gdf.plot(ax=ax, color='red', markersize =1)
pednet.plot(ax=ax,color='black', markersize =1, alpha = 0.1)
plt.title('Walking Distance To Closest Coffee Shop')

# getting limits
ax.set(xlim=(-79.65, -79.1), ylim=(43.55, 43.85))

##### Visualizing with Mapbox to Create Interactive Map

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(df, lat="lat", lon="lng", hover_name="shop_name", hover_data=["address"],
                        color="id", zoom=9, height=300)
fig.update_layout(mapbox_style="streets", 
                  mapbox=dict(accesstoken=mapbox_access_token))
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# help(px.scatter_mapbox)